In [169]:
import os
os.environ["PATH"] += os.pathsep + "/usr/bin/dot"

In [170]:
from keras.models import Sequential
from keras.layers import Embedding, Dense, Flatten
from keras.preprocessing import text
from keras.utils import to_categorical
from keras.preprocessing import sequence
# from keras.utils import np_utils
import numpy as np



In [171]:
file_path='C:/Users/ASUS/Desktop/corona.txt'
# data=open('.\corona.txt','r')

In [172]:
with open(file_path, 'r') as file:
    data = file.read()

In [173]:
print(data)

The speed of transmission is an important point of difference between the two viruses. Influenza has a shorter median incubation period (the time from infection to appearance of symptoms) and a shorter serial interval (the time between successive cases) than COVID-19 virus. The serial interval for COVID-19 virus is estimated to be 5-6 days, while for influenza virus, the serial interval is 3 days. This means that influenza can spread faster than COVID-19. 

Further, transmission in the first 3-5 days of illness, or potentially pre-symptomatic transmission â€“transmission of the virus before the appearance of symptoms â€“ is a major driver of transmission for influenza. In contrast, while we are learning that there are people who can shed COVID-19 virus 24-48 hours prior to symptom onset, at present, this does not appear to be a major driver of transmission. 

The reproductive number â€“ the number of secondary infections generated from one infected individual â€“ is understood to be be

In [174]:
tokenizer = text.Tokenizer()
tokenizer.fit_on_texts(data)
word2id = tokenizer.word_index

In [175]:
word2id['PAD'] = 0
id2word = {v:k for k, v in word2id.items()}
wids = [[word2id[w] for w in text.text_to_word_sequence(doc)] for doc in data]




In [176]:
vocab_size = len(word2id)
embed_size = 100
window_size = 2 # context window size

In [177]:
print('Vocabulary Size:', vocab_size)
print('Vocabulary Sample:', list(word2id.items())[:10])

Vocabulary Size: 37
Vocabulary Sample: [('e', 1), ('i', 2), ('t', 3), ('s', 4), ('r', 5), ('n', 6), ('o', 7), ('a', 8), ('f', 9), ('d', 10)]


In [178]:
def generate_context_word_pairs(corpus, window_size, vocab_size):
    context_length = window_size*2
    for words in corpus:
        sentence_length = len(words)
        for index, word in enumerate(words):
            context_words = []
            label_word   = []            
            start = index - window_size
            end = index + window_size + 1
            
            context_words.append([words[i] 
                                 for i in range(start, end) 
                                 if 0 <= i < sentence_length 
                                 and i != index])
            label_word.append(word)

            x = sequence.pad_sequences(context_words,
            maxlen=context_length)
            y =to_categorical(label_word, vocab_size)
            yield (x, y)
            
            
# Test this out for some samples
i = 0
for x, y in generate_context_word_pairs(corpus=wids, 
window_size=window_size, vocab_size=vocab_size):
    if 0 not in x[0]:
        print('Context (X):',[id2word[w] for w in x[0]],'-> Target (Y):',id2word[np.argwhere(y[0])[0][0]])
    
        if i == 10:
            break
        i += 1

In [179]:
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense, Embedding, Lambda

# build CBOW architecture
cbow = Sequential()
cbow.add(Embedding(input_dim=vocab_size, output_dim=embed_size, input_length=window_size*2))
cbow.add(Lambda(lambda x: K.mean(x, axis=1), output_shape=(embed_size,)))
cbow.add(Dense(vocab_size, activation='softmax'))
cbow.compile(loss='categorical_crossentropy', optimizer='rmsprop')

# view model summary
print(cbow.summary())

# visualize model structure
from IPython.display import SVG
# from tensorflow.keras.utils.vis_utils import model_to_dot
# Assuming your model is named 'model'
from tensorflow.keras.utils import plot_model

# plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)




Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_13 (Embedding)    (None, 4, 100)            3700      
                                                                 
 lambda_13 (Lambda)          (None, 100)               0         
                                                                 
 dense_13 (Dense)            (None, 37)                3737      
                                                                 
Total params: 7437 (29.05 KB)
Trainable params: 7437 (29.05 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [180]:

    for epoch in range(1, 6):
        loss = 0.
        i = 0
        for x, y in generate_context_word_pairs(corpus=wids, window_size=window_size, vocab_size=vocab_size):
            i += 1
            loss += cbow.train_on_batch(x, y)
            if i % 100000 == 0:
                print('Processed {} (context, word) pairs'.format(i))
    

In [181]:
weights = cbow.get_weights()[0]
weights = weights[1:]
print(weights.shape)

import pandas as pd
pd.DataFrame(weights, index=list(id2word.values())[1:]).head()

(36, 100)


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
i,0.021014,-0.029548,-0.019853,0.035607,-0.046591,-0.021184,0.034601,0.044587,0.020664,-0.032072,...,0.049776,0.048546,0.041240,-0.036389,-0.040024,0.022411,-0.023450,0.033564,-0.047312,0.009969
t,0.035466,-0.026246,0.005481,0.021203,-0.019324,0.025819,0.023532,-0.022167,-0.017751,-0.028721,...,0.042374,0.049647,-0.022741,-0.028646,-0.043170,0.020829,-0.000013,0.018079,-0.044271,0.004267
s,0.021251,-0.022198,-0.023825,-0.000849,0.003993,-0.026476,0.047260,-0.018484,0.022855,-0.041076,...,-0.023941,0.021548,-0.013312,0.027140,0.004451,0.033044,-0.021514,0.004945,0.019152,-0.040694
r,0.012779,0.006544,-0.029650,0.017467,0.027381,-0.002858,-0.029681,-0.010581,-0.014890,0.046050,...,0.002921,-0.041814,-0.043920,0.012217,0.019987,-0.004679,0.001593,0.035177,0.046905,0.049222
n,0.022822,0.022840,-0.003912,0.034875,0.036993,-0.002496,-0.013619,-0.037596,0.029687,0.035699,...,0.046956,-0.031854,0.044524,-0.012778,0.027245,-0.008200,-0.007962,0.026210,-0.018419,-0.049042


In [182]:
from sklearn.metrics.pairwise import euclidean_distances

# compute pairwise distance matrix
distance_matrix = euclidean_distances(weights)
print(distance_matrix.shape)

# view contextually similar words
similar_words = {search_term: [id2word[idx] for idx in distance_matrix[word2id[search_term]-1].argsort()[1:6]+1] 
                   for search_term in ['influenza', 'covid']}

similar_words

(36, 36)


KeyError: 'influenza'